# POSTTEST 5 Kecerdasan Buatan
## Tommy Candra Gunawan (2209106068)

#### perintah 


### Import Library

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, callbacks
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

### Membaca Database

In [2]:
df = pd.read_csv('laptop_pricedataset.csv')



# Memisahkan Fitur dan Target
X = df.drop(['Price (Euro)'], axis=1)
y = df['Price (Euro)']

# Membagi Dataset menjadi Training Set dan Test Set (70% Training, 30% Testing)

### Mengecek missing values

In [3]:
df.isnull().sum()

Company                0
Product                0
TypeName               0
Inches                 0
ScreenResolution       0
CPU_Company            0
CPU_Type               0
CPU_Frequency (GHz)    0
RAM (GB)               0
Memory                 0
GPU_Company            0
GPU_Type               0
OpSys                  0
Weight (kg)            0
Price (Euro)           0
dtype: int64

### Mengecek Nilai Duplikat

In [4]:
duplicate = df.duplicated().sum()
print('Jumlah nilai duplikat pada data: ', duplicate)

Jumlah nilai duplikat pada data:  0


### Menampilkan Tipe Data Setiap Kolom

In [5]:
print(df.dtypes)


Company                 object
Product                 object
TypeName                object
Inches                 float64
ScreenResolution        object
CPU_Company             object
CPU_Type                object
CPU_Frequency (GHz)    float64
RAM (GB)                 int64
Memory                  object
GPU_Company             object
GPU_Type                object
OpSys                   object
Weight (kg)            float64
Price (Euro)           float64
dtype: object


### Menghapus kolom yg tipenya object

In [6]:
df = df.drop(['Company'], axis=1)
df = df.drop(['Product'], axis=1)
df = df.drop(['TypeName'], axis=1)
df = df.drop(['ScreenResolution'], axis=1)
df = df.drop(['CPU_Company'], axis=1)
df = df.drop(['CPU_Type'], axis=1)
df = df.drop(['Memory'], axis=1)
df = df.drop(['GPU_Company'], axis=1)
df = df.drop(['GPU_Type'], axis=1)
df = df.drop(['OpSys'], axis=1)

### splitting data

In [7]:
X = df.drop(['Price (Euro)'], axis=1)
y = df['Price (Euro)']

# Membagi dataset menjadi training set dan test set (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### menormalisasi fitur dan target

In [8]:
mm = MinMaxScaler()
X_train = mm.fit_transform(X_train)
X_test = mm.transform(X_test)

In [9]:
y_train = mm.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = mm.transform(np.array(y_test).reshape(-1, 1))

In [10]:
X_train.shape

(892, 4)

### definisi model dan komplikasi model

In [11]:
model = Sequential([
    layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]], kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(1)
])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

c:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### callback EarlyStopping

In [12]:
es = callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

In [13]:
history = model.fit(
    X_train, y_train,
    epochs = 50,
    validation_split=0.2,
    callbacks=[es]
)

Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.8619 - mean_absolute_error: 0.5784 - val_loss: 0.1732 - val_mean_absolute_error: 0.1153
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2260 - mean_absolute_error: 0.2295 - val_loss: 0.1557 - val_mean_absolute_error: 0.0955
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2021 - mean_absolute_error: 0.1874 - val_loss: 0.1505 - val_mean_absolute_error: 0.0910
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1767 - mean_absolute_error: 0.1530 - val_loss: 0.1478 - val_mean_absolute_error: 0.0900
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1666 - mean_absolute_error: 0.1400 - val_loss: 0.1485 - val_mean_absolute_error: 0.1031
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1630 - mean_absolute_error: 0.1360 - val_loss: 0.1488 - val_mean_absolute_error: 0.1108
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1567 - mean_absolute_error: 0.1337 - val_loss: 0.14

### Membagi data training menjadi training penuh dan validation set

In [15]:

X_train_full, X_val, y_train_full, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Melatih ulang model tanpa menggunakan validation_split

In [16]:

history = model.fit(
    X_train_full, y_train_full,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[es]
)

Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1676 - mean_absolute_error: 0.1413 - val_loss: 0.1543 - val_mean_absolute_error: 0.1132
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1648 - mean_absolute_error: 0.1395 - val_loss: 0.1531 - val_mean_absolute_error: 0.1215
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1509 - mean_absolute_error: 0.1184 - val_loss: 0.1555 - val_mean_absolute_error: 0.1339
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1478 - mean_absolute_error: 0.1167 - val_loss: 0.1536 - val_mean_absolute_error: 0.1349
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1424 - mean_absolute_error: 0.1066 - val_loss: 0.1543 - val_mean_absolute_error: 0.1387
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.


### Evaluasi untuk model training

In [17]:

train_loss, train_mae = model.evaluate(X_train_full, y_train_full)
val_loss, val_mae = model.evaluate(X_val, y_val)

print(f"Training MAE: {train_mae*100:.2f}%")
print(f"Validation MAE: {val_mae*100:.2f}%")

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1502 - mean_absolute_error: 0.1035
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1540 - mean_absolute_error: 0.1091 
Training MAE: 10.32%
Validation MAE: 11.32%
